In [1]:
import pickle

In [125]:
d_b = pickle.load(open('dataset_11194_basic.pkl', 'rb'))

In [126]:
set([o['type'] for o in d_b])

{'answer', 'guess', 'question_pick', 'questions_candidates'}

In [127]:
for o in d_b:
    if o['type'] != "answer":
        continue
    o['prompt'] = o['prompt'].replace("""Format your answer like this: First, reason about your answer for 1-3 sentences. If the answer is obvious - state it. If not - reason about whether YES or NO would be more helpful and truthful.
Then, your last line should be the answer, only YES or NO.
Don't output anything else.""", """Format your answer like this: Your last and only line should be the answer, only YES or NO.
Don't output anything else.""")
    if "reason" in o['prompt'].lower():
        print("!!!")
        break
    o['response'] = o['response'].split("\n")[-1].strip().replace("Answer: ", "").lower()
    if o['response'].lower() not in ['yes', 'no']:
        print("!", o['response'])
        break

In [128]:
for o in d_b:
    if o['type'] != "guess":
        continue
    o['prompt'] = o['prompt'].replace("""Format your answer like this: First, reason about your decision for 2-4 sentences. Summarize what you know, summarize space of possibilities, etc.
Then, your last line should be the keywords that you want to guess.
Don't output anything else. Only output the keywords that you want to guess. Don't say anything like "My guess is {keyword}" - always just "{keyword}\"""", 
              """Format your answer like this: only output the keyword that you want to guess, nothing else. Don't say anything like "My guess is {{keyword}}" - always just "{{keyword}}\"""")
    if "sentences" in o['prompt'].lower():
        print("!!!")
    o['response'] = o['response'].split("\n")[-1].strip().replace("Answer: ", "").replace("Is it an ", "").replace("Is it a ", "").replace("Is it in", "").replace("?", "").replace("\"", "")
    # if any symbol is not alphabetical  -output the response
    if any([not c.isalnum() and c != " " and c != "-" and c != "." and c != "," and c != "'" for c in o['response']]):
        print("!", o['response'])

In [129]:
d_b = [o for o in d_b if (o['type'] != "question_pick") or ("?" in o['response'])]
for o in d_b:
    if o['type'] != "question_pick":
        continue
    o['prompt'] = o['prompt'].replace("""You goal is to pick the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.

In any case, try to make reason about what decision is the best here, balance risk, and choose the best next question. Act as a competetive extremely smart player that you are.

Don't forget that some inconsistencies in answers can be caused by errors in the answering LLM.

Format your answer like this: First, reason about your decision for 2-4 sentences. Summarize what you know, summarize space of possibilities, reason about what question would be the most beneficial.
Then, your last line should be the questions that you've picked.
Don't output anything else.""", 
                          """You goal is to choose the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.

You should keep balance of safe questions and more creative and non-orthodox ones.
Safe questions are usually ones that split the space of possibilities in half.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.
Consider asking non-standard questions like "Does the keyword consist of a single word?, however only do it if you don't know it yet, and if you think that it's important to know at this point in the game.
Consider asking questions like "Does the keyword start with letter 'A'?" or "Does the keyword contain letter 'E'?". However, don't overdo it, focus on more substantial questions.

In any case, try to make reason about what decision is the best here, balance risk, and choose the best next question. Act as a competetive extremely smart player that you are.

Don't forget that some inconsistencies in answers can be caused by errors in the answering LLM.

Format your answer like this: Your last and only line should be the question that you want to ask. Just output the question, not "Question: ..." or "I pick question..."
Don't output anything else!""")
    if "sentences" in o['prompt']:
        print("!!!")
        break
    o['response'] = o['response'].split("\n")[-1].strip().split(":")[-1]
    if "?" not in o['response'] or ":" in o['response'] or "my" in o['response'].lower() or "i " in o['response'].lower()  or "here " in o['response'].lower():
        print("!", o['response'])

! Is it related to space or astronomy?
! Is it related to space or astronomy?
! Is it related to space or astronomy?
! Is it related to space or astronomy?
! Is it related to space or astronomy?
! Is it related to space or astronomy?
! Is it related to space or astronomy?
! Is it located in Saudi Arabia?
! Is the city located in Abu Dhabi Emirate?
! Is it a peak in the Nanda Devi range?
! Is it a peak in the Gangotri range?
! Is it a tributary of the Mississippi River?
! Is it located in a state that borders the Mississippi River?
! Is it related to the anatomy of mammals?


In [130]:
d_b = [o for o in d_b if not o['type'] == "questions_candidates"]

In [131]:
pickle.dump(d_b, open('!final_dataset_11194_basic.pkl', 'wb'))

In [132]:
d_b = pickle.load(open("dataset_6016_with_hints_after_n_step.pkl", 'rb'))

In [133]:
set([o['type'] for o in d_b])

{'answer',
 'guess',
 'question_pick',
 'question_pick_with_answer',
 'questions_candidates',
 'questions_candidates_with_answer'}

In [134]:
for o in d_b:
    if o['type'] != "answer":
        continue
    o['prompt'] = o['prompt'].replace("""Format your answer like this: First, reason about your answer for 1-3 sentences. If the answer is obvious - state it. If not - reason about whether YES or NO would be more helpful and truthful.
Then, your last line should be the answer, only YES or NO.
Don't output anything else.""", """Format your answer like this: Your last and only line should be the answer, only YES or NO.
Don't output anything else.""")
    if "reason" in o['prompt'].lower():
        print("!!!")
        break
    o['response'] = o['response'].split("\n")[-1].strip().replace("Answer: ", "").lower()
    if o['response'].lower() not in ['yes', 'no']:
        print("!", o['response'])
        break
    break

In [135]:
print(o['response'])

no


In [136]:
for o in d_b:
    if o['type'] != "guess":
        continue
    o['prompt'] = o['prompt'].replace("""Format your answer like this: First, reason about your decision for 2-4 sentences. Summarize what you know, summarize space of possibilities, etc.
Then, your last line should be the keywords that you want to guess.
Don't output anything else. Only output the keywords that you want to guess. Don't say anything like "My guess is {keyword}" - always just "{keyword}\"""", 
              """Format your answer like this: only output the keyword that you want to guess, nothing else. Don't say anything like "My guess is {{keyword}}" - always just "{{keyword}}\"""")
    if "sentences" in o['prompt'].lower():
        print("!!!")
    o['response'] = o['response'].split("\n")[-1].strip().replace("Answer: ", "").replace("Is it an ", "").replace("Is it a ", "").replace("Is it in", "").replace("?", "").replace("\"", "")
    # if any symbol is not alphabetical  -output the response
    if any([not c.isalnum() and c != " " and c != "-" and c != "." and c != "," and c != "'" for c in o['response']]):
        print("!", o['response'])

! Aoraki / Mount Cook
! Aoraki / Mount Cook


In [137]:
d_b = [o for o in d_b if (o['type'] != "question_pick" and o['type'] != "question_pick_with_answer") or ("?" in o['response'])]
for o in d_b:
    if o['type'] != "question_pick" or o['type'] != "question_pick_with_answer":
        continue
    o['prompt'] = o['prompt'].replace("""You goal is to pick the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.

In any case, try to make reason about what decision is the best here, balance risk, and choose the best next question. Act as a competetive extremely smart player that you are.

Don't forget that some inconsistencies in answers can be caused by errors in the answering LLM.

Format your answer like this: First, reason about your decision for 2-4 sentences. Summarize what you know, summarize space of possibilities, reason about what question would be the most beneficial.
Then, your last line should be the questions that you've picked.
Don't output anything else.""", 
                          """You goal is to choose the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.

You should keep balance of safe questions and more creative and non-orthodox ones.
Safe questions are usually ones that split the space of possibilities in half.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.
Consider asking non-standard questions like "Does the keyword consist of a single word?, however only do it if you don't know it yet, and if you think that it's important to know at this point in the game.
Consider asking questions like "Does the keyword start with letter 'A'?" or "Does the keyword contain letter 'E'?". However, don't overdo it, focus on more substantial questions.

In any case, try to make reason about what decision is the best here, balance risk, and choose the best next question. Act as a competetive extremely smart player that you are.

Don't forget that some inconsistencies in answers can be caused by errors in the answering LLM.

Format your answer like this: Your last and only line should be the question that you want to ask. Just output the question, not "Question: ..." or "I pick question..."
Don't output anything else!""")
    if "sentences" in o['prompt']:
        print("!!!")
        break
    o['response'] = o['response'].split("\n")[-1].strip().split(":")[-1]
    if "?" not in o['response'] or ":" in o['response'] or "my" in o['response'].lower() or "i " in o['response'].lower()  or "here " in o['response'].lower():
        print("!", o['response'])

In [138]:
d_b = [o for o in d_b if not o['type'] == "questions_candidates" and not o['type'] == "questions_candidates_with_answer"]

In [140]:
set([o['type'] for o in d_b])

{'answer', 'guess', 'question_pick', 'question_pick_with_answer'}

In [141]:
pickle.dump(d_b, open('!final_dataset_6016_with_hints_after_n_step.pkl', 'wb'))

In [143]:
d_b = pickle.load(open("dataset_17476_from_snapshots_fixed.pkl", 'rb'))

In [145]:
set([o['type'] for o in d_b])

{'guess_from_snapshot', 'questions_candidates_from_snapshot'}

In [153]:
for o in d_b:
    if o['type'] != "guess_from_snapshot":
        continue
    o['prompt'] = o['prompt'].replace("""Format your answer like this: First, reason about your decision for 2-4 sentences. Summarize what you know, summarize space of possibilities, etc.
Then, your last line should be the keyword that you want to guess.
Don't output anything else. Only output the keyword that you want to guess. Don't say anything like "My guess is {keyword}" - always just "{keyword}\"""", 
              """Format your answer like this: only output the keyword that you want to guess, nothing else. Don't say anything like "My guess is {{keyword}}" - always just "{{keyword}}\"""")
    if "sentences" in o['prompt'].lower():
        print("!!!")
    o['response'] = o['response'].split("\n")[-1].strip().replace("Answer: ", "").replace("Is it an ", "").replace("Is it a ", "").replace("Is it in", "").replace("?", "").replace("\"", "")
    # if any symbol is not alphabetical  -output the response
    if any([not c.isalnum() and c != " " and c != "-" and c != "." and c != "," and c != "'" for c in o['response']]):
        print("!", o['response'])

In [164]:
d_b = [o for o in d_b if (o['type'] != "question_pick" and o['type'] != "question_pick_with_answer") or ("?" in o['response'])]
for o in d_b:
    if o['type'] != "questions_candidates_from_snapshot":
        continue
    o['prompt'] = o['prompt'].replace(
    """You need to generate 1 candidates for the next question to ask.
You should mix safe questions with more creative and non-orthodox ones.
Safe questions are usually ones that split the space of possibilities in half.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.
Consider asking non-standard questions like "Does the keyword consist of a single word?, however only do it if you don't know it yet, and if you think that it's important to know at this point in the game.
Consider asking questions like "Does the keyword start with letter 'A'?" or "Does the keyword contain letter 'E'?". However, don't overdo it, focus on more substantial questions.

Format your answer like this: output every question on a new line. Don't output anything else.""",
    """You goal is to choose the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.

You should keep balance of safe questions and more creative and non-orthodox ones.
Safe questions are usually ones that split the space of possibilities in half.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.
Consider asking non-standard questions like "Does the keyword consist of a single word?, however only do it if you don't know it yet, and if you think that it's important to know at this point in the game.
Consider asking questions like "Does the keyword start with letter 'A'?" or "Does the keyword contain letter 'E'?". However, don't overdo it, focus on more substantial questions.

In any case, try to make reason about what decision is the best here, balance risk, and choose the best next question. Act as a competetive extremely smart player that you are.

Don't forget that some inconsistencies in answers can be caused by errors in the answering LLM.

Format your answer like this: Your last and only line should be the question that you want to ask. Just output the question, not "Question: ..." or "I pick question..."
Don't output anything else!"""
)
    if "sentences" in o['prompt']:
        print("!!!")
        break
    o['response'] = o['response'].split("\n")[-1].strip().split(":")[-1]
    if "?" not in o['response'] or ":" in o['response'] or "my" in o['response'].lower() or "i " in o['response'].lower()  or "here " in o['response'].lower():
        print("!", o['response'])
    o['type'] = "question_pick_from_snapshot"  # TODO enable line

In [165]:
pickle.dump(d_b, open('!final_dataset_17476_from_snapshots_fixed.pkl', 'wb'))

In [92]:
d_b = pickle.load(open("!final_dataset_17476_from_snapshots_fixed.pkl", 'rb'))

In [93]:
import pandas as pd

In [94]:
df = pd.DataFrame(d_b)

In [95]:
df['length'] = df['prompt'].apply(len)
df['has_candidates'] = df['prompt'].apply(lambda x: "candidates" in x.lower())

In [96]:
df.has_candidates.sum()

0

In [48]:
df = df.sort_values('length', ascending=False)

In [66]:
print(df.iloc[0]['prompt'])

You are playing "20 Questions" game.
    

    
Here are some candidates for the next question:
Is it a place?
Is it a thing?
Is it a concept?
Is it something you can touch?
Is it used in everyday life?
Is it larger than a car?
Is it found indoors?
Is it related to technology?
Is it made of metal?
Is it a single word?

You goal is to pick the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.

In any case, try to make reason about what decision is the best here, balance risk, and choose the best next question. Act as a competetive extremely smart player that you are.

Don't forget that some inconsistencies in answers can be caused by errors in the answering LLM.

Format your answer like this: First, reason about your decision for 2-4 sentences. Summarize what you know, su

type
answer           2808
question_pick    2807
guess            2765
Name: count, dtype: int64

In [66]:
d_b = pickle.load(open("!final_dataset_11194_basic.pkl", 'rb'))

In [78]:
for o in d_b:
    if o['type'] != "question_pick":
        continue
    to_delete = o['prompt'][o['prompt'].index("Here are some candidates for the next question"):o['prompt'].index("You goal is to pick the best next question")]
    o['prompt'] = o['prompt'].replace(to_delete, "")
    if "candidates" in o['prompt']:
        print("!!!")
        break

In [79]:
for o in d_b:
    o['prompt'] = o['prompt'].replace("You goal", "Your goal")

In [80]:
pickle.dump(d_b, open('!final_dataset_11194_basic.pkl', 'wb'))

In [85]:
d_b = pickle.load(open("!final_dataset_6016_with_hints_after_n_step.pkl", 'rb'))

In [86]:
for o in d_b:
    if o['type'] != "question_pick" and o['type'] != "question_pick_with_answer":
        continue
    to_delete = o['prompt'][o['prompt'].index("Here are some candidates for the next question"):o['prompt'].index("You goal is to pick the best next question")]
    o['prompt'] = o['prompt'].replace(to_delete, "")
    if "candidates" in o['prompt']:
        print("!!!")
        break

In [87]:
for o in d_b:
    o['prompt'] = o['prompt'].replace("You goal", "Your goal")

In [88]:
set([o['type'] for o in d_b])

{'answer', 'guess', 'question_pick', 'question_pick_with_answer'}

In [89]:
pickle.dump(d_b, open('!final_dataset_6016_with_hints_after_n_step.pkl', 'wb'))

In [97]:
d_b = pickle.load(open("!final_dataset_17476_from_snapshots_fixed.pkl", 'rb'))

In [98]:
for o in d_b:
    o['prompt'] = o['prompt'].replace("You goal", "Your goal")

In [99]:
pickle.dump(d_b, open('!final_dataset_17476_from_snapshots_fixed.pkl', 'wb'))